In [1]:
import pyhf
import json
import numpy as np

import simplify

# for uncertainties on fit results, I think we still need minuit as optimiser (performance isn't a problem for this tool)
pyhf.set_backend(pyhf.tensorlib, "minuit")

# Get the workspace from model spec

spec = json.load(
    open(
        "test/EwkOneLeptonTwoBjets2018/BkgOnly.json", "r"
    )
)

In [2]:
original_ws = pyhf.Workspace(spec) # origin ws from full LH
pruned_ws = original_ws.prune(channels=['SRLMEM_mct2','SRMMEM_mct2','SRHMEM_mct2']) # prune out SRs from original ws
no_staterror_ws = original_ws.prune(modifiers=['staterror_SRLMEM_mct2','staterror_SRMMEM_mct2','staterror_SRHMEM_mct2']) # prune out staterror from SRs, but leave SRs in place for now

In [3]:
# get model and data for each ws we just created
pruned_model = pruned_ws.model(modifier_settings = {"normsys": {"interpcode": "code4"},"histosys": {"interpcode": "code4p"},},poi_name = None)
pruned_data = pruned_ws.data(pruned_model)
original_model = original_ws.model(modifier_settings = {"normsys": {"interpcode": "code4"},"histosys": {"interpcode": "code4p"},},poi_name = None)
original_data = original_ws.data(original_model)
no_staterror_model = no_staterror_ws.model(modifier_settings = {"normsys": {"interpcode": "code4"},"histosys": {"interpcode": "code4p"},},poi_name = None)
no_staterror_data = no_staterror_ws.data(no_staterror_model)

In [ ]:
# run fit for every ws
pruned_fit_result = simplify.fitter.fit((pruned_model,pruned_data))
original_fit_result = simplify.fitter.fit((original_model,original_data))
no_staterror_fit_result = simplify.fitter.fit((no_staterror_model,no_staterror_data))

In [9]:
yields = simplify.yields.get_yields((original_model,original_data), original_fit_result)
yields

Yields(regions=[&#39;SRHMEM_mct2&#39;, &#39;SRLMEM_mct2&#39;, &#39;SRMMEM_mct2&#39;, &#39;STCREM_cuts&#39;, &#39;TRHMEM_cuts&#39;, &#39;TRLMEM_cuts&#39;, &#39;TRMMEM_cuts&#39;, &#39;WREM_cuts&#39;], samples=[&#39;diboson&#39;, &#39;multiboson&#39;, &#39;singletop&#39;, &#39;ttbar&#39;, &#39;tth&#39;, &#39;ttv&#39;, &#39;vh&#39;, &#39;wjets&#39;, &#39;zjets&#39;], yields={&#39;SRHMEM_mct2&#39;: array([[0.0632191 , 0.07246621, 0.10172258],
       [0.        , 0.        , 0.        ],
       [3.75302015, 2.65783516, 0.50767731],
       [0.51750445, 0.17511258, 0.24253512],
       [0.02887177, 0.02100665, 0.02574003],
       [0.24845997, 0.1753172 , 0.2087737 ],
       [0.0091588 , 0.00875104, 0.00628611],
       [1.81969673, 1.50564236, 0.5112493 ],
       [0.        , 0.        , 0.        ]]), &#39;SRLMEM_mct2&#39;: array([[3.82052204e-01, 1.35593506e-01, 1.94599462e-01],
       [0.00000000e+00, 1.45691835e-03, 2.90519926e-03],
       [4.61457081e+00, 8.81572109e-01, 8.42905564e-01],
  

In [10]:
# Correlation matrix
plt = simplify.plot.correlation_matrix(original_fit_result,"test/figures/",pruning_threshold=0.15)


In [11]:
# Pull plot
plt = simplify.plot.pulls(original_fit_result,"test/figures/")


[102 101 100  99  98  97  96  95  94  93  92  91  90  89  88  87  86  85
  84  83  82  81  80  79  78  77  76  75  74  73  72  71  70  69  68  67
  66  65  64  63  62  61  60  59  58  57  56  55  54  53  52  51  50  49
  48  47  46  45  44  43  42  41  40  39  38  37  36  35  34  33  32  31
  30  29  28  27  26  25  24  23  22  21  20  19  18  17  16  15  14  13
  12  11  10   9   8   7   6   5   4   3   2   1   0]
[119 118 117 116 115 114 113 112 111 110 109 108 107 106 105 104 103]


In [12]:
tables = simplify.plot.yieldsTable(spec, "test/figures/", original_fit_result)

In [ ]:
stdevs = simplify.model_utils.calculate_stdev(model,fit_result.bestfit,fit_result.uncertainty,fit_result.corr_mat)
